In [ ]:
# Set up BigQuery client
bigquery_client = bigquery.Client(credentials=credentials)

# Define BigQuery dataset name
dataset_id = 'pokemon_dataset'

# Create a BigQuery dataset if it doesn't exist
dataset_ref = bigquery_client.dataset(dataset_id)
dataset = bigquery.Dataset(dataset_ref)
try:
    bigquery_client.get_dataset(dataset)
except Exception as e:
    bigquery_client.create_dataset(dataset)

# Define BigQuery table schema for each dimension
entity_schema = [
    bigquery.SchemaField('Entity_ID', 'INTEGER', mode='REQUIRED'),
    bigquery.SchemaField('Name', 'STRING'),
]

type_schema = [
    bigquery.SchemaField('Type_ID', 'INTEGER', mode='REQUIRED'),
    bigquery.SchemaField('Type_Name', 'STRING'),
]

ability_schema = [
    bigquery.SchemaField('Ability_ID', 'INTEGER', mode='REQUIRED'),
    bigquery.SchemaField('Ability_Name', 'STRING'),
]

# Load data into BigQuery tables
for schema, table_name in zip([entity_schema, type_schema, ability_schema], ['EntityTable', 'TypeTable', 'AbilityTable']):
    table_ref = bigquery_client.dataset(dataset_id).table(table_name)
    job_config = bigquery.LoadJobConfig(schema=schema, skip_leading_rows=1, source_format=bigquery.SourceFormat.CSV)
    with open(csv_filename, 'rb') as source_file:
        job = bigquery_client.load_table_from_file(source_file, table_ref, job_config=job_config)

    # Wait for the job to complete
    job.result()

# Define BigQuery table schema for the Fact dimension
fact_schema = [
    bigquery.SchemaField('Entity_ID', 'INTEGER'),
    bigquery.SchemaField('Type_ID', 'INTEGER'),
    bigquery.SchemaField('HP_Stat', 'INTEGER'),
    bigquery.SchemaField('Attack_Stat', 'INTEGER'),
    bigquery.SchemaField('Defense_Stat', 'INTEGER'),
    bigquery.SchemaField('Special_Attack_Stat', 'INTEGER'),
    bigquery.SchemaField('Special_Defense_Stat', 'INTEGER'),
    bigquery.SchemaField('Speed_Stat', 'INTEGER'),
    bigquery.SchemaField('Abilities', 'STRING'),
    bigquery.SchemaField('Total_Base_Stat', 'INTEGER'),
]

# Load data into the Fact table
fact_table_ref = bigquery_client.dataset(dataset_id).table('FactTable')
fact_job_config = bigquery.LoadJobConfig(schema=fact_schema, skip_leading_rows=1, source_format=bigquery.SourceFormat.CSV)
with open(csv_filename, 'rb') as source_file:
    try:
        fact_job = bigquery_client.load_table_from_file(source_file, fact_table_ref, job_config=fact_job_config)
        # Wait for the job to complete
        fact_job.result()
    except Exception as e:
        print(f"Error loading data into FactTable: {str(e)}")
        for error in fact_job.errors:
            print(error)

# Print the URLs of the created tables
for table_name in ['EntityTable', 'TypeTable', 'AbilityTable', 'FactTable']:
    print(f'Table created in BigQuery: {bigquery_client.project}.{dataset_id}.{table_name}')
